In [49]:
import pandas as pd
import panel as pn
import plotly.express as px
from plotly.graph_objects import Figure, Scatter, Bar
import numpy as np

In [50]:
pn.extension("plotly")

In [51]:
df = pd.read_csv('./data/eduwa.csv')
df.head()

,Unnamed: 0,NCES.School.ID,State.School.ID,NCES.District.ID,State.District.ID,Low.Grade,High.Grade,School.Name,District,County,...,Phone,Locale.Code,LocaleType,LocaleSub,Charter,Title.I.School,Title.1.School.Wide,Student.Teacher.Ratio,Free.Lunch,Reduced.Lunch
0,1,530486002475,WA-31025-1656,5304860,WA-31025,6,8,10th Street School,Marysville School District,Snohomish,...,(360)965-0400,22.0,Suburb,Suburb: Midsize,No,Yes,Yes,23.4,28.0,3.0
1,2,530270001270,WA-06114-1646,5302700,WA-06114,KG,12,49th Street Academy,Evergreen School District (Clark),Clark,...,(360)604-6700,12.0,City,City: Midsize,No,No,NaN,8.4,53.0,9.0
2,3,530910002602,WA-34033-4500,5309100,WA-34033,9,12,A G West Black Hills High School,Tumwater School District,Thurston,...,(360)709-7800,13.0,City,City: Small,No,No,NaN,21.5,169.0,40.0
3,4,530003000001,WA-14005-2834,5300030,WA-14005,PK,6,A J West Elementary,Aberdeen School District,Grays Harbor,...,(360)538-2131,33.0,Town,Town: Remote,No,Yes,Yes,15.9,292.0,10.0
4,5,530825002361,WA-32081-1533,5308250,WA-32081,9,12,A-3 Multiagency Adolescent Prog,Spokane School District,Spokane,...,(509)354-6299,12.0,City,City: Midsize,No,No,NaN,6.5,12.0,4.0


In [52]:
newdf = df[["County","Reduced.Lunch"]]
reducedLunchCounty = newdf.groupby(['County']).sum().sort_values(by="Reduced.Lunch",ascending=False).reset_index()
reducedLunchCounty['Cumulative'] = 100 * (reducedLunchCounty["Reduced.Lunch"].cumsum()/reducedLunchCounty["Reduced.Lunch"].sum())
reducedLunchCounty

,County,Reduced.Lunch,Cumulative
0,King,16048.0,20.842912
1,Pierce,11210.0,35.402299
2,Snohomish,8087.0,45.905578
3,Clark,6388.0,54.202221
4,Spokane,5889.0,61.850770
5,Thurston,3162.0,65.957530
6,Kitsap,2998.0,69.851289
7,Benton,2412.0,72.983960
8,Whatcom,2003.0,75.585428
9,Yakima,1613.0,77.680369


In [53]:
def graph_pareto(dataframe, col):
    df = dataframe.copy()

    data = [
        Bar(
          name = "Count",  
          x = df.County,
          y = df[f'{col}'], 
          marker= {"color": list(np.repeat('rgb(71, 71, 135)', 5)) + list(np.repeat('rgb(112, 111, 211)', len(df.index) - 5))}
        ),
        Scatter(
          line= {
            "color": "rgb(192, 57, 43)", 
            "width": 3
          }, 
          name = "Percentage", 
          x = df.County,
          y = df['Cumulative'], 
          yaxis= "y2",
          mode='lines+markers'
        ),
    ]

    layout = {
      # Title Graph
      "title": {
        'text': "Reduced Lunch by County<br><span style='font-size:15px; color: rgb(100, 100, 100);'>Reduced Lunch Individuals in Schools in Washington State</span>",
        'font': dict(size=30),
      },
      
      # Source/Caption
      "annotations": [{
        'xref': 'paper',
        'yref': 'paper',
        'x': 1, 
        'y': -0.5,
        'showarrow': False,
        'text': '<span style="font-size:12px; color: gray; font-family: Courier New, monospace;">Source: National Center for Education Statistics</span>',
        'align': 'right'
      }],
      
      # Font 
      "font": {
        "size": 14, 
        "color": "rgb(44, 44, 84)", 
        "family": "Times New Roman, monospace"
      },

      # Graph Box 
      "margin": {
        "b": 120, 
        "l": 50, 
        "r": 50, 
        "t": 100,
      }, 
      "height": 400, 

      # Graph Box 
      "plot_bgcolor": "rgb(255, 255, 255)", 


      # Settings Legend
      "legend": {
        "x": 0.79, 
        "y": 1.2, 
        "font": {
          "size": 12, 
          "color": "rgb(44, 44, 84)", 
          "family": "Courier New, monospace"
        },
        'orientation': 'h',
      },
      # Xaxis
      "xaxis": {
        "showline": True,
        "linecolor": "rgb(0, 0, 0)",
        "linewidth": 2,
        "ticks": "outside",
        "tickwidth": 2, 
        "tickcolor": 'rgb(0, 0, 0)',
        "range": [(-1),len(reducedLunchCounty.index)]
      },
      # Yaxis 1 position left

      "yaxis": {
        "title": "Count Reduced Lunch",
        "titlefont": {
        "size": 16,
        "color": "rgb(71, 71, 135)", 
        "family": "Courier New, monospace"
        },
        "showline": True,
        "linecolor": "rgb(0, 0, 0)",
        "linewidth": 2,
        "ticks": "outside",
        "tickwidth": 2, 
        "tickcolor": 'rgb(0, 0, 0)',
        #"automargin": True
        #"shift": -1
      }, 


      # Yaxis 2 position right
      "yaxis2": {
        "side": "right",
        "range": [0, 100], 
        "title": "Percentage of Total Reduced Lunch",
        "titlefont": {
          "size": 16, 
          "color": "rgb(71, 71, 135)", 
          "family": "Courier New, monospace"
        },
        "showline": True,
        "linecolor": "rgb(0, 0, 0)",
        "linewidth": 2,
        "overlaying": "y",
        "ticksuffix": " %",
        "ticks": "outside",
        "tickwidth": 2, 
        "tickcolor": 'rgb(0, 0, 0)',
        #"automargin": True
        #"shift": 1
      }, 
    }

    # Build Graph
    fig = Figure(data=data, layout=layout)
    fig.update_layout(autosize=True)
    # Show Graph
    fig.show()
    
    
    plotPane = pn.pane.Plotly(fig)
    source_annotation = pn.pane.Markdown(
        '<span style="font-size: 12px; color: gray;">Source: National Center for Education Statistics</span>',
        width=600, height=40, align="start"
    )

    # Combine Plotly graph and source annotation in a layout
    layout = pn.Column(plotPane, source_annotation, styles={"border": "1px solid black"})

    return layout
lunchfig = graph_pareto(reducedLunchCounty, 'Reduced.Lunch')

In [54]:
dfarrests = pd.read_excel('./data/MSP DFS Arrests 19-20Q1.xlsx', sheet_name='MSP DFS Arrests')
dfcodes = pd.read_excel('./data/MSP DFS Arrests 19-20Q1.xlsx', sheet_name='Codes', usecols="A:B", skiprows=74)

In [55]:
dfarrests = dfarrests[['Arrest Type','Age']].dropna()
dfarrests.head()

,Arrest Type,Age
0,W,25.0
1,M,26.0
2,M,28.0
3,M,28.0
4,M,28.0


In [56]:
dfcodes.columns = ['Arrest Type', 'Long Form']
dfcodes.head()

,Arrest Type,Long Form
0,F,Felony
1,J,Juvenile
2,M,Misdemeanor
3,W,Warrant
4,O,Other


In [57]:
mergeddf = dfarrests.merge(dfcodes, on='Arrest Type', how='inner')
mergeddf.head()

,Arrest Type,Age,Long Form
0,W,25.0,Warrant
1,M,26.0,Misdemeanor
2,M,28.0,Misdemeanor
3,M,28.0,Misdemeanor
4,M,28.0,Misdemeanor


In [58]:
figviol = px.violin(
    mergeddf,
    x = "Age",
    y = "Long Form",
    box = True,
    points="all",
    color = "Long Form",
    range_x = [0, None],
    labels = {
        "Long Form" : "Arrest Type",
        "Age" : "Arrestee Age (Years)"
    },
    title = "Felony Arrests are Younger than other Arrests in Massachusetts<br><span style='font-size:12px; color: rgb(100, 100, 100);'>Age Distribution of Arrests in Massachusetts from January 2019 to March 2020 </span>"
)
figviol.update_layout(
    showlegend=False
)
figviol.show()

In [59]:
arrestSourceAnnotation = pn.pane.Markdown(
        '<span style="font-size: 12px; color: gray;">Source: Massachusetts Executive Office of Public Safety and Security</span>',
        width=600, height=40, align="start"
    )
arrestsfig = pn.Column(figviol,arrestSourceAnnotation, styles={"border": "1px solid black"})

In [60]:
dash = pn.Column(lunchfig,arrestsfig,sizing_mode="stretch_width")

In [61]:
#server = dash.show()

In [62]:
from bokeh.resources import INLINE
from pathlib import Path

dash.save(f"{Path.home()}/Documents/School/690V/index.html", resources=INLINE, embed=True)